# Desambiguación de significados

### Introducción
La intención del proyecto es identificar diferentes significados en los textos para observar si así se puede extraer más información de los textos. Para esto, lo que se va a realizar es una modificación del texto asignando un identificador a las palabras que se han desambiguado con la notación `palabra_n_codigo`. 

Debido a que no existe una librería que realice la desambiguación tal y como se requiere en este proyecto, se ha decidido crear un algoritmo de desambiguación apoyándose en la librería `WordNet` de nltk. Esta librería contiene un diccionario de palabras, para cada palabra almacena su acepción y sus hiperónimos, de forma que se crea un árbol con las palabras en inglés.

### Solución
Dado que los árboles de palabras en `WordNet` están divididos por tipos de palabras, en este proyecto para acotar el problema se ha decidido desambiguar los sustantivos. El algoritmo desarrollado se apoya en la métrica JCN. La librería permite obtener acepciones posibles para un término en inglés. Se ha utilizado esta característica para comparar todas las acepciones de los sustantivos 2 a 2 y así obtener las definiciones de palabras que estén más relacionadas, asumiendo que es más probable que sean las correctas. Una vez obtenida la desambiguación, se sustituyen los sustantivos por los identificadores que ofrece `WordNet` para cada sustantivo.

#### Exploración de otras posibilidades
Antes de llegar a esta solución se han creado algoritmos basados en otras métricas. Un ejemplo se encuentra en el archivo `Disambiguation_cosine_distance` donde se ha intentado crear un algoritmo que calcula la similitud entre definiciones de sustantivos basado en la distancia coseno.
Otro intento se ha basado en simplemente dar por hecho que las definiciones con el hiperónimo más cercano tienen más relación, pero ha dado peores resultados que la métrica JNC.

#### Medida de similitud JNC
La similitud de JCN (Jiang-Conrath) es una medida de similitud semántica entre dos palabras en un espacio vectorial. Esta métrica se basa en la idea de que palabras que comparten información más específica tienen una similitud semántica más alta. La fórmula para calcular la similitud de JCN entre dos palabras \(w_1\) y \(w_2\) es la siguiente:

$$
JCN(w_1, w_2) = \frac{1}{JCN\_distance(w_1, w_2) + 1}
$$

Donde \(JCN\_distance(w_1, w_2)\) es la distancia de Jiang-Conrath entre las dos palabras, y se calcula de la siguiente manera:

$$
JCN\_distance(w_1, w_2) = IC(w_1) + IC(w_2) - 2 \times IC(LCA(w_1, w_2))
$$
Aquí, los términos utilizados son:

- \(IC(w)\): La información de contenido de la palabra \(w\). Esta medida se basa en la frecuencia de aparición de la palabra en un corpus y se utiliza para cuantificar la cantidad de información que proporciona la palabra. Cuanto menos frecuente sea una palabra, más información proporciona y, por lo tanto, su \(IC\) es mayor.

- \(LCA(w_1, w_2)\): El ancestro común más bajo de las palabras \(w_1\) y \(w_2\) en una jerarquía de conceptos, en este caso la de WordNet.


In [2]:
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk import pos_tag, word_tokenize

#### Cálculo de la similitud
A continuación el algoritmo que calcula las acepciones que más se parezcan para 2 sustantivos dados.

In [3]:
# Cargar la información de contenido de WordNet
brown_ic = wordnet_ic.ic('ic-brown.dat')

def calculate_jcn_similarity(word1, word2):
    # Obtener los synsets para cada palabra
    synsets_word1 = wn.synsets(word1, pos=wn.NOUN)
    synsets_word2 = wn.synsets(word2, pos=wn.NOUN)

    max_similarity = -1
    best_synset_word1 = None
    best_synset_word2 = None

    # Calcular la similitud JCN entre los synsets de las dos palabras
    for synset1 in synsets_word1:
        for synset2 in synsets_word2:
            similarity = synset1.jcn_similarity(synset2, brown_ic)
            if similarity > max_similarity:
                max_similarity = similarity
                best_synset_word1 = synset1
                best_synset_word2 = synset2
                
    if best_synset_word1 is None : best_synset_word1 = word1
    if best_synset_word2 is None : best_synset_word2 = word2
    
    return best_synset_word1, best_synset_word2, max_similarity

### Desambiguación de textos
Para realizar la desambiguación primero se tokeniza el texto y se clasifican las palabras según su clase, esto se utilizará para obtener los sustantivos y guardar las posiciones del texto en las que luego irán los identificadores.  

Una vez obtenidos los sustantivos, se procede a recorrer la lista que los contiene, comparando siempre una palabra con la siguiente en el texto. Para estas palabras se obtiene las acepciones que más relación tengan. El algoritmo tiene flexibilidad para corregir una acepción cuando encuentra otra que tiene más relación. Es decir, para 3 palabras `A`, `B` y `C`, el algoritmo:  
-  Compara `A` y `B`, obteniendo los significados que tengan más relación.
-  Compara `B` y `C` para obtener el significado de la palabra `C`, pero si existe una acepción de `B` que se parezca más a `C` que a `A`, se actualiza.

Cuando se termina la desambiguación, se identifican los sustantivos que se han desambiguado y se cambian en el texto por el identificador de `WordNet`, con el formato `palabra_n_codigo`

In [4]:
def disambiguateText(text, verbose_ = False):
    # Tokenizado
    tokens = word_tokenize(text)
    # Etiquetado y obtención de substantivos
    pos_tags = pos_tag(tokens)
    if verbose_: print(pos_tags)
    nouns = [word for word, tag in pos_tags if ((tag == 'NN' or tag == ' NNS') and len(wn.synsets('word', pos=wn.NOUN)) > 0)]
    if verbose_: print('Nouns to Disambiguate: ', nouns)
    #Inicializamos variables de apoyo
    last_noun = None
    last_sim = -1
    toret_nouns = []
    #Desambiguación del texto
    for i in range(len(nouns)):
        if i < len(nouns)-1:
            if verbose_: print('Disambiguate: ',nouns[i],' vs ',nouns[i+1])
            first_noun, second_noun, similarity = calculate_jcn_similarity(nouns[i], nouns[i+1])
            if verbose_: print(first_noun, ' ', second_noun, ' ', similarity, ' ', last_sim)
            if similarity >= last_sim:
                toret_nouns.append(first_noun)
            else:
                toret_nouns.append(last_noun)
            last_sim = similarity
            last_noun = second_noun
        else:
            second_noun, first_noun, similarity = calculate_jcn_similarity(nouns[i], nouns[i-1])
            toret_nouns.append(second_noun)
            if verbose_: print(second_noun, ' ', first_noun)
    if verbose_:
        print(toret_nouns)
        for i in toret_nouns:
            if type(i) is str:
                print(i)
            else:
                print(i.name(),'\n',i.definition())
    # Substitución de las palabras desambiguadas en el texto

    for i in range(len(nouns)):
        index = tokens.index(nouns[i])
        if type(toret_nouns[i]) is str:
            tokens[index] = toret_nouns[i]
        else:
            tokens[index] = toret_nouns[i].name()
    if verbose_: print(tokens)        
    return ' '.join(tokens)

#### Ejemplo de uso

In [8]:
example = "John withdrew cash from the bank and fished by the river."
disambiguateText(example, verbose_=True)

[('John', 'NNP'), ('withdrew', 'VBD'), ('cash', 'NN'), ('from', 'IN'), ('the', 'DT'), ('bank', 'NN'), ('and', 'CC'), ('fished', 'VBN'), ('by', 'IN'), ('the', 'DT'), ('river', 'NN'), ('.', '.')]
Nouns to Disambiguate:  ['cash', 'bank', 'river']
Disambiguate:  cash  vs  bank
Synset('cash.n.02')   Synset('bank.n.05')   0.08397598296431394   -1
Disambiguate:  bank  vs  river
Synset('bank.n.07')   Synset('river.n.01')   0.07049299927013367   0.08397598296431394
Synset('river.n.01')   Synset('bank.n.07')
[Synset('cash.n.02'), Synset('bank.n.05'), Synset('river.n.01')]
cash.n.02 
 prompt payment for goods or services in currency or by check
bank.n.05 
 a supply or stock held in reserve for future use (especially in emergencies)
river.n.01 
 a large natural stream of water (larger than a creek)
['John', 'withdrew', 'cash.n.02', 'from', 'the', 'bank.n.05', 'and', 'fished', 'by', 'the', 'river.n.01', '.']


'John withdrew cash.n.02 from the bank.n.05 and fished by the river.n.01 .'

#### Tratado del texto
Para realizar la desambiguación, primero se eliminan las stopwords y caracteres que no sean letras o espacios, luego se realiza la desambiguación y se almacena.

El identificador original que aporta `WordNet` para los sustantivos tiene la forma `palabra.n.codigo`, pero los puntos dan problemas a la hora de aplicar un vectorizer y se separa el identificador en `palabra`, `n` y `codigo`, perdiendo así la desambiguación, por lo que se ha decidido cambiar `.` por `_` para evitar el problema. 

In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords

df = pd.read_json('dataTest.json')
stop_words = set(stopwords.words('english'))
disambiguatedText = df['text'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word.lower() not in stop_words]))
disambiguatedText = disambiguatedText.apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
disambiguatedText = disambiguatedText.apply(disambiguateText)
disambiguatedText = disambiguatedText.apply(lambda x: re.sub(r'\.n\.', '_n_', x))
disambiguatedText

0       marc_n_01 east_n_01 fiuczynski south_n_03 home...
1       document_n_01 information_n_01 server_n_03 des...
2       last modified wednesday 20 nov 96 07 36 47 gre...
3       c_n_10 537 programming assignment_n_02 three_n...
4       kurt partridge_n_01 kurt partridge academic in...
                              ...                        
1654    c_n_10 110 section_n_01 2 late policy_n_01 lat...
1655    last modified saturday 03 declination_n_03 94 ...
1656    last modified saturday 12 oct 96 07 33 01 gree...
1657    last modified tuesday 19 nov 96 19 34 50 gmt c...
1658    nancy hall computer_n_01 sciences department_n...
Name: text, Length: 1659, dtype: object

In [16]:
df['text'] = disambiguatedText

In [12]:
df.to_json('disambiguated_dataTest.json')